# load data

In [8]:
import torch
from torch_geometric.data import Data
from torch_geometric.loader import DataLoader
import os.path as osp
from tqdm import tqdm

In [5]:
N = 6111
dir_name = './data/'
datasets = []
for i in tqdm(range(N)):
	data = torch.load(osp.join(dir_name,f'data_{i}.pt'))
	datasets.append(data)

100%|██████████| 6111/6111 [01:19<00:00, 76.83it/s] 


In [99]:
for i, data in enumerate(datasets):
	data.edge_index = data.edge_index.type(torch.long)
	torch.save(data, osp.join(dir_name,f'data_{i}.pt'))

In [6]:
datasets[0]

Data(x=[327, 86], edge_index=[2, 6233], edge_attr=[6233, 5])

## train-test split

In [105]:
train_idx = list()
test_name = list()
test_idx = list()
y_train = list()
with open('./data/raw/graph_labels.txt', 'r') as f:
    for i,line in enumerate(f):
        t = line.split(',')
        if len(t[1][:-1]) == 0:
            test_name.append(t[0])
            test_idx.append(i)
        else:
            train_idx.append(i)
            y_train.append(int(t[1][:-1]))

In [106]:
len(train_idx),len(test_idx)

(4888, 1223)

In [107]:
train_val_set = [datasets[i] for i in train_idx]
test_set = [datasets[i] for i in test_idx]

## add label

In [108]:
train_set[0].y

6

In [109]:
for data, y in zip(train_val_set, y_train):
	data.y = y

## train-valid split

In [110]:
train_set, val_set = torch.utils.data.random_split(train_val_set, [0.9,0.1], generator=torch.Generator().manual_seed(42))

In [111]:
len(train_set), len(val_set)

(4400, 488)

## Hyperparameters

In [171]:
import numpy as np
num_node_features = train_set[0].num_node_features # 86
print(num_node_features)
num_classes = len(np.unique(y_train)) # 18
print(num_classes)
lr = 0.001
epochs = 100
batch_size = 64
hidden_dim = 64

86
18


## Dataloader

In [139]:
from torch_geometric.loader import DataLoader
from torch.utils.data.dataloader import default_collate
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
collate_fn = lambda x: tuple(x_.to(device) for x_ in default_collate(x))
train_loader = DataLoader(train_set, batch_size=64, shuffle=True, collate_fn=collate_fn)
val_loader = DataLoader(val_set, batch_size=64, shuffle=True, collate_fn=collate_fn)
test_loader = DataLoader(test_set, batch_size=64, shuffle=False, collate_fn=collate_fn)

In [170]:
train_val_loader =DataLoader(train_val_set, batch_size=64, shuffle=True, collate_fn=collate_fn)

In [140]:
for step, data in enumerate(train_loader):
	print(f'Step {step + 1}:')
	print('=======')
	print(f'Number of graphs in the current batch: {data.num_graphs}')
	print(data.is_cuda)
	print()
	break

Step 1:
Number of graphs in the current batch: 64
False



# GNN0: baseline

In [115]:
import torch.nn as nn
import torch.nn.functional as F
import torch_geometric.nn as gnn
from torch_geometric.utils import dropout_node

In [154]:
from torch import embedding


class GCN_0(torch.nn.Module):
    def __init__(self, hidden_channels):
        self.p = 0.2
        super(GCN_0, self).__init__()
        torch.manual_seed(12345)
        self.conv1 = gnn.GCNConv(num_node_features, hidden_channels)
        self.conv2 = gnn.GCNConv(hidden_channels, hidden_channels)
        self.bn = gnn.BatchNorm(hidden_channels)
        self.dropout = nn.Dropout(self.p)
        self.lin1 = nn.Linear(hidden_channels, hidden_channels)
        self.lin2 = nn.Linear(hidden_channels, num_classes)

    def forward(self, x, edge_index, batch):
        # 1. Obtain node embeddings 
        x = self.conv1(x, edge_index)
        x = x.relu()
        edge_index = dropout_node(edge_index)[0] # edge_index, edge_mask, node_mask
        x = self.conv2(x, edge_index)
        x = x.relu()

        # 2. Readout layer
        x = gnn.global_add_pool(x, batch)  # [batch_size, hidden_channels]
        x = self.bn(x)
        x = x.relu()
        embedding = x

        # 3. Apply a final classifier
        x = self.lin1(x)
        x = x.relu()
        
        x = F.dropout(x, p=self.p, training=self.training)
        x = self.lin2(x)
        
        return F.log_softmax(x, dim=1), embedding

In [163]:
model_0 = GCN_0(hidden_channels=hidden_dim).to(device)
print(model_0)
optimizer_0 = torch.optim.Adam(model_0.parameters(), lr=lr)
criterion = nn.CrossEntropyLoss()

GCN_0(
  (conv1): GCNConv(86, 64)
  (conv2): GCNConv(64, 64)
  (bn): BatchNorm(64)
  (dropout): Dropout(p=0.2, inplace=False)
  (lin1): Linear(in_features=64, out_features=64, bias=True)
  (lin2): Linear(in_features=64, out_features=18, bias=True)
)


In [181]:
def train(model, optimizer):
	model.train()
	for data in tqdm(train_loader):
		data = data.to(device)
		out,_ = model(data.x, data.edge_index, data.batch)
		loss = criterion(out, data.y)
		loss.backward()
		optimizer.step()
		optimizer.zero_grad()

def eval(model, loader):
	model.eval()
	correct = 0
	for data in loader:
		data = data.to(device)
		out,_ = model(data.x, data.edge_index, data.batch)
		loss = criterion(out, data.y)
		pred = out.argmax(dim=1)
		correct += int((pred == data.y).sum())
		acc = correct / len(loader.dataset)
	return acc, loss

def predict(model):
	pred_proba = []
	model.eval()
	for data in test_loader:
		data = data.to(device)
		out,_ = model(data.x, data.edge_index, data.batch)
		pred_proba.append(out)
	pred_proba = torch.cat(pred_proba, dim=0)
	pred_proba = torch.exp(pred_proba)
	return pred_proba.detach().cpu().numpy()

In [176]:
import os.path as osp
def load_checkpoint(model, optimizer, path):
	if osp.exists(path):
		checkpoint = torch.load(path)
		model.load_state_dict(checkpoint['model_state_dict'])
		optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
		print('checkpoint loaded')

In [169]:
import time
# pbar = tqdm(range(epochs))
for epoch in range(epochs):
	train(model_0, optimizer_0)
	train_acc, train_loss = eval(model_0, train_loader)
	val_acc, val_loss = eval(model_0, val_loader)
	time.sleep(0.5)
	print(f'Epoch: {epoch:03d}, Train Loss: {train_loss:.4f}, Train Acc: {train_acc:.4f}, Val Loss: {val_loss:.4f}, Val Acc: {val_acc:.4f}')


100%|██████████| 69/69 [00:05<00:00, 11.95it/s]


Epoch: 000, Train Loss: 1.7957, Train Acc: 0.5325, Val Loss: 1.8774, Val Acc: 0.4713


100%|██████████| 69/69 [00:03<00:00, 19.72it/s]


Epoch: 001, Train Loss: 1.5660, Train Acc: 0.5255, Val Loss: 2.1350, Val Acc: 0.4406


100%|██████████| 69/69 [00:03<00:00, 18.70it/s]


...

Epoch: 061, Train Loss: 1.4734, Train Acc: 0.5900, Val Loss: 1.8023, Val Acc: 0.4570


100%|██████████| 69/69 [00:03<00:00, 17.88it/s]


Epoch: 062, Train Loss: 1.2535, Train Acc: 0.5984, Val Loss: 38.4535, Val Acc: 0.4754


 30%|███       | 21/69 [00:01<00:02, 18.28it/s]


KeyboardInterrupt: 

In [ ]:
# PATH = './model/model_0.pt'
# torch.save({
#             'epoch': epochs,
#             'model_state_dict': model_0.state_dict(),
#             'optimizer_state_dict': optimizer_0.state_dict(),
#             }, PATH)

training sample model for over 150 epochs, no significant improvement. we keep the 100 epochs checkpoint.

retrain model together with validation data

In [172]:
model_0_full = GCN_0(hidden_channels=hidden_dim).to(device)
print(model_0_full)
optimizer_0_full = torch.optim.Adam(model_0_full.parameters(), lr=lr)

GCN_0(
  (conv1): GCNConv(86, 64)
  (conv2): GCNConv(64, 64)
  (bn): BatchNorm(64)
  (dropout): Dropout(p=0.2, inplace=False)
  (lin1): Linear(in_features=64, out_features=64, bias=True)
  (lin2): Linear(in_features=64, out_features=18, bias=True)
)


In [173]:
for epoch in range(epochs):
	train(model_0_full, optimizer_0_full)
	train_acc, train_loss = eval(model_0_full, train_val_loader)
	time.sleep(0.5)
	print(f'Epoch: {epoch:03d}, Train Loss: {train_loss:.4f}, Train Acc: {train_acc:.4f}')

100%|██████████| 69/69 [00:03<00:00, 18.34it/s]


Epoch: 000, Train Loss: 2.5891, Train Acc: 0.2361


100%|██████████| 69/69 [00:03<00:00, 20.54it/s]


Epoch: 001, Train Loss: 2.0847, Train Acc: 0.2287


100%|██████████| 69/69 [00:03<00:00, 20.08it/s]


...

Epoch: 097, Train Loss: 1.5849, Train Acc: 0.5072


100%|██████████| 69/69 [00:03<00:00, 18.87it/s]


Epoch: 098, Train Loss: 2.2545, Train Acc: 0.4988


100%|██████████| 69/69 [00:03<00:00, 19.08it/s]


Epoch: 099, Train Loss: 2.1586, Train Acc: 0.5125


In [175]:
PATH = './model/model_0_full.pt'
torch.save({
            'epoch': epochs,
            'model_state_dict': model_0_full.state_dict(),
            'optimizer_state_dict': optimizer_0_full.state_dict(),
            }, PATH)

In [177]:
PATH = './model/model_0.pt'
PATH_full = './model/model_0_full.pt'
load_checkpoint(model_0, optimizer_0, PATH)
load_checkpoint(model_0_full, optimizer_0_full, PATH_full)

checkpoint loaded
checkpoint loaded


In [185]:
import csv
def write_csv(fname, pred_proba):
    with open(f'./prediction/{fname}.csv', 'w') as csvfile:
        writer = csv.writer(csvfile, delimiter=',')
        lst = list()
        for i in range(18):
            lst.append('class'+str(i))
        lst.insert(0, "name")
        writer.writerow(lst)
        for i, protein in enumerate(test_name):
            lst = pred_proba[i,:].tolist()
            lst.insert(0, protein)
            writer.writerow(lst)

In [186]:
pred_proba_0 = predict(model_0)
write_csv('gnn0',pred_proba_0)


In [187]:
write_csv('gnn0_full', predict(model_0_full))